In [1]:
import numpy as np
np.random.seed(8)
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(rescale=1./255)
                                   

train_generator = train_datagen.flow_from_directory(
        'C:/Users/Owner2/Desktop/deep/train',
        target_size=(150, 150),
        batch_size=64,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'C:/Users/Owner2/Desktop/deep/test',
        target_size=(150, 150),    
        batch_size=16,
        class_mode='categorical')

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

Found 6400 images belonging to 8 classes.
Found 1600 images belonging to 8 classes.


In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

from keras.layers import Dropout

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(8, activation='softmax'))

model.summary()

W1128 16:44:59.532709 12000 deprecation_wrapper.py:119] From C:\Users\Owner2\Anaconda3\envs\keras2\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1128 16:44:59.549779 12000 deprecation_wrapper.py:119] From C:\Users\Owner2\Anaconda3\envs\keras2\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1128 16:44:59.551789 12000 deprecation_wrapper.py:119] From C:\Users\Owner2\Anaconda3\envs\keras2\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1128 16:44:59.566794 12000 deprecation_wrapper.py:119] From C:\Users\Owner2\Anaconda3\envs\keras2\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 36992)             0         
__________

In [4]:
from keras import backend as K
import tensorflow as tf
from sklearn.metrics import recall_score 

from sklearn.metrics import precision_score 

from sklearn.metrics import f1_score 
from keras.optimizers import adam
from keras import metrics
from tensorflow.python.keras.callbacks import TensorBoard

def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision

def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score


In [7]:
Xtrain,ytrain=next(train_generator)
Xtest,ytest=next(test_generator)

In [8]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

In [9]:
# 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=50,
        validation_data=test_generator,
        validation_steps=150)

W1128 16:45:27.839650 12000 deprecation_wrapper.py:119] From C:\Users\Owner2\Anaconda3\envs\keras2\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/50
100/100 [==============================] - 116s 1s/step - loss: 0.1995 - acc: 0.9081 - f1_m: 0.5568 - precision_m: 0.6595 - recall_m: 0.5008 - val_loss: 0.1571 - val_acc: 0.9223 - val_f1_m: 0.6772 - val_precision_m: 0.7023 - val_recall_m: 0.6554
Epoch 2/50
100/100 [==============================] - 108s 1s/step - loss: 0.1465 - acc: 0.9295 - f1_m: 0.7070 - precision_m: 0.7355 - recall_m: 0.6811 - val_loss: 0.1608 - val_acc: 0.9226 - val_f1_m: 0.6784 - val_precision_m: 0.7035 - val_recall_m: 0.6567
Epoch 3/50
100/100 [==============================] - 107s 1s/step - loss: 0.1352 - acc: 0.9382 - f1_m: 0.7454 - precision_m: 0.7672 - recall_m: 0.7253 - val_loss: 0.1457 - val_acc: 0.9354 - val_f1_m: 0.7358 - val_precision_m: 0.7512 - val_recall_m: 0.7221
Epoch 4/50
100/100 [==============================] - 107s 1s/step - loss: 0.1152 - acc: 0.9462 - f1_m: 0.7811 - precision_m: 0.7938 - recall_m: 0.7691 - val_loss: 0.1481 - val_acc: 0.9332 - val_f1_m: 0.7276 - val_precision_m: 0.

Epoch 33/50
100/100 [==============================] - 106s 1s/step - loss: 1.0586e-04 - acc: 0.9999 - f1_m: 0.9997 - precision_m: 0.9997 - recall_m: 0.9997 - val_loss: 0.4222 - val_acc: 0.9400 - val_f1_m: 0.7598 - val_precision_m: 0.7604 - val_recall_m: 0.7592
Epoch 34/50
100/100 [==============================] - 106s 1s/step - loss: 7.9575e-05 - acc: 1.0000 - f1_m: 0.9998 - precision_m: 0.9998 - recall_m: 0.9998 - val_loss: 0.4448 - val_acc: 0.9372 - val_f1_m: 0.7486 - val_precision_m: 0.7490 - val_recall_m: 0.7483
Epoch 35/50
100/100 [==============================] - 106s 1s/step - loss: 9.6899e-05 - acc: 1.0000 - f1_m: 0.9998 - precision_m: 0.9998 - recall_m: 0.9998 - val_loss: 0.4345 - val_acc: 0.9385 - val_f1_m: 0.7538 - val_precision_m: 0.7544 - val_recall_m: 0.7533
Epoch 36/50
100/100 [==============================] - 107s 1s/step - loss: 9.8442e-05 - acc: 1.0000 - f1_m: 0.9998 - precision_m: 0.9998 - recall_m: 0.9998 - val_loss: 0.4382 - val_acc: 0.9382 - val_f1_m: 0.7524 -

In [15]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(Xtest, 
                                                             ytest,verbose=0)

print(f1_score)
print(precision)
print(recall)

0.8749999403953552
0.875
0.875


In [18]:
# 모델 평가하기
print("-- Evaluate --")

scores = model.evaluate_generator(
            test_generator, 
            steps = 5)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
acc: 95.62%


In [13]:
# 모델 예측하기
print("-- Predict --")

prediction = model.predict_generator(
            test_generator, 
            steps = 5)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(prediction)

-- Predict --
[[0.011 0.989 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 1.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 1.000 0.000]
 [0.638 0.362 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.984 0.000 0.000 0.011 0.005]
 [0.003 0.997 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.000 1.000]
 [0.888 0.112 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 1.000 0.000]
 [0.000 0.000 0.393 0.000 0.607 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.489 0.511]
 [0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000]
 [0.001 0.999 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.000 1.000]
 [0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000]
 [1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.015 0.000 0.000 0.000 0.985]
 [0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.0

In [14]:
model.save('endoscope_model_ver.95.31.h5')